In [36]:
import psycopg2
import pandas as pd
import pandas.io.sql as sqlio
from config import database, user, password, host, port

con = psycopg2.connect(database=database, user=user, password=password, host=host, port=port)
sql = "SELECT * FROM Players;"
all_df = sqlio.read_sql_query(sql, con)
con = None

In [37]:
import warnings
warnings.filterwarnings('ignore')

#### Postgresql DB connection 
    we will display the dataframe now that we have connected to the postgresql with our notebook and we will display it through pandas.

In [38]:
all_df.head()

,name,age,year,team,league,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete
0,Bobby Abreu,26,2000,PHI,NL,4354977,0.994676,6.213081,0,0.401235,69700927,33.333332
1,Bobby Abreu,27,2001,PHI,NL,7193330,1.029782,5.039903,0,0.530864,60144837,66.666664
2,Bobby Abreu,28,2002,PHI,NL,9000338,1.001989,5.818425,0,0.493827,82364470,100.000000
3,Bobby Abreu,29,2003,PHI,NL,12643905,1.016617,5.282226,0,0.530864,98344579,16.666668
4,Bobby Abreu,30,2004,PHI,NL,14346025,1.042947,6.289872,0,0.530864,126162692,33.333332


#### Columns and Target Variable
    We will create a list of the columns that are useful for our data as well as pick our target column - which is the WAR (Wins above Replacement) since that is our predictor variable.

In [39]:
columns = ["name", "age", "year", "team", "division", "league", "player_salary", "percent_season_played", "num_dl_movements", "team_win_percentage", 
           "team_payroll", "percent_contract_complete"]
          

target = ["war_psp"]


##### Variables to encode: team & league

    Loading in the data and cleaning it will be the next steps in this analysis. 

    We are first going to encode the categorical variables team and league in order to use them in our regression analysis.

    This will allow us to separate the data into testing and training sets. This is a resampling method. 

In [40]:
league_binary_encoded = pd.get_dummies(all_df, columns=["league"])
league_binary_encoded.head()

,name,age,year,team,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete,league_AL,league_AL/NL,league_NL,league_NL/AL,league_NL/AL/AL,league_NL/AL/NL
0,Bobby Abreu,26,2000,PHI,4354977,0.994676,6.213081,0,0.401235,69700927,33.333332,0,0,1,0,0,0
1,Bobby Abreu,27,2001,PHI,7193330,1.029782,5.039903,0,0.530864,60144837,66.666664,0,0,1,0,0,0
2,Bobby Abreu,28,2002,PHI,9000338,1.001989,5.818425,0,0.493827,82364470,100.000000,0,0,1,0,0,0
3,Bobby Abreu,29,2003,PHI,12643905,1.016617,5.282226,0,0.530864,98344579,16.666668,0,0,1,0,0,0
4,Bobby Abreu,30,2004,PHI,14346025,1.042947,6.289872,0,0.530864,126162692,33.333332,0,0,1,0,0,0


In [41]:
league_binary_encoded.loc[(league_binary_encoded["league_NL/AL/NL"] == 1)]

,name,age,year,team,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete,league_AL,league_AL/NL,league_NL,league_NL/AL,league_NL/AL/AL,league_NL/AL/NL
320,David DeJesus,33,2013,CHC/TBR/WSN,4664132,0.642151,2.958806,1,0.502058,100091461,66.666664,0,0,0,0,0,1


In [42]:
league_binary_encoded.loc[(league_binary_encoded["league_NL/AL/AL"] == 1)]

,name,age,year,team,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete,league_AL,league_AL/NL,league_NL,league_NL/AL,league_NL/AL/AL,league_NL/AL/NL
1731,Jaime Garcia,30,2017,ATL/MIN/NYY,12515845,0.839572,1.465032,0,0.510288,142025719,100.0,0,0,0,0,1,0


    These two players are the only ones that have been traded 2 times within a single season over the last 25 season, therefore there is not a significance for keeping these two players in our dataset. Removing two players' data is not going to change our results significantly enough due to the large sample size.

In [43]:
MLB_df = all_df.loc[(all_df["league"] != "NL/AL/NL") & (all_df["league"] != "NL/AL/AL")]


In [44]:
MLB_df.loc[(all_df["league"] == "NL/AL/AL") | (all_df["league"] == "NL/AL/NL")]

,name,age,year,team,league,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete


In [45]:
MLB_df["league"].unique().tolist()

['NL', 'AL/NL', 'AL', 'NL/AL']

In [46]:
league_binary_encoded.loc[(league_binary_encoded["league_NL/AL"] == 1)].count()

name                         39
age                          39
year                         39
team                         39
player_salary                39
percent_season_played        39
war_psp                      39
num_dl_movements             39
team_win_percentage          39
team_payroll                 39
percent_contract_complete    39
league_AL                    39
league_AL/NL                 39
league_NL                    39
league_NL/AL                 39
league_NL/AL/AL              39
league_NL/AL/NL              39
dtype: int64

In [47]:
league_binary_encoded.loc[(league_binary_encoded["league_AL/NL"] == 1)].count()

name                         43
age                          43
year                         43
team                         43
player_salary                43
percent_season_played        43
war_psp                      43
num_dl_movements             43
team_win_percentage          43
team_payroll                 43
percent_contract_complete    43
league_AL                    43
league_AL/NL                 43
league_NL                    43
league_NL/AL                 43
league_NL/AL/AL              43
league_NL/AL/NL              43
dtype: int64

    To make the data more uniform - we are going to redefine all of the single trades in a season to be Uniform in AL/NL instead of both AL/NL and NL/AL 

In [48]:
MLB_df = MLB_df.replace(to_replace = "NL/AL", value="AL/NL", regex=True)



In [49]:
MLB_df["league"].unique().tolist()

['NL', 'AL/NL', 'AL']

    above we have shown that the new all_df we will be working with has a uniform league column now - with no NL/AL values but with replaced AL/NL values. We have also reassured the absence of the multiple trades players. 
    
    Now we will create the encoded values for the different leagues 
    
    0 -> NL
    1 -> AL
    2 -> AL/NL
    
    and convert the values to the encoded values 

In [50]:
MLB_df = MLB_df.replace({'league': {'NL': 0, 'AL': 1, 'AL/NL': 2}})
MLB_df.head()

,name,age,year,team,league,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete
0,Bobby Abreu,26,2000,PHI,0,4354977,0.994676,6.213081,0,0.401235,69700927,33.333332
1,Bobby Abreu,27,2001,PHI,0,7193330,1.029782,5.039903,0,0.530864,60144837,66.666664
2,Bobby Abreu,28,2002,PHI,0,9000338,1.001989,5.818425,0,0.493827,82364470,100.000000
3,Bobby Abreu,29,2003,PHI,0,12643905,1.016617,5.282226,0,0.530864,98344579,16.666668
4,Bobby Abreu,30,2004,PHI,0,14346025,1.042947,6.289872,0,0.530864,126162692,33.333332


    Now we are going to view the different team names in this dataset:

In [51]:
team_names = MLB_df["team"].unique().tolist()
len(team_names)

167

    there are 167 unique team names / trades in our current dataframe
    
    from here we need to either encode the values or determine if the team name plays a imperative role in our analysis.

In [52]:
team_names_dict = {team_names[i]: i for i in range(len(team_names))}

    from the team_names_dict we will conver the team values to their encoded values that they are given by the dictionary 

In [53]:
MLB_df = MLB_df.replace({'team': team_names_dict})
MLB_df.head()

,name,age,year,team,league,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete
0,Bobby Abreu,26,2000,0,0,4354977,0.994676,6.213081,0,0.401235,69700927,33.333332
1,Bobby Abreu,27,2001,0,0,7193330,1.029782,5.039903,0,0.530864,60144837,66.666664
2,Bobby Abreu,28,2002,0,0,9000338,1.001989,5.818425,0,0.493827,82364470,100.000000
3,Bobby Abreu,29,2003,0,0,12643905,1.016617,5.282226,0,0.530864,98344579,16.666668
4,Bobby Abreu,30,2004,0,0,14346025,1.042947,6.289872,0,0.530864,126162692,33.333332


    Within considering the 167 unique team values - we have decided to remove the team column from the Machine Learning model due to simplification purposes

In [54]:
MLB_df = MLB_df.drop(["team"], axis=1)
MLB_df = MLB_df.set_index(["name"])

    Setting the Name column as the index allows us to remove it as a categorical variable from our dataset while keeping the names of the players in the dataset for distinguishablity. 
    We removed the team columns for the reasons explained above.

In [55]:
MLB_df

,age,year,league,player_salary,percent_season_played,war_psp,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete
name,,,,,,,,,,
Bobby Abreu,26,2000,0,4354977,0.994676,6.213081,0,0.401235,69700927,33.333332
Bobby Abreu,27,2001,0,7193330,1.029782,5.039903,0,0.530864,60144837,66.666664
Bobby Abreu,28,2002,0,9000338,1.001989,5.818425,0,0.493827,82364470,100.000000
Bobby Abreu,29,2003,0,12643905,1.016617,5.282226,0,0.530864,98344579,16.666668
Bobby Abreu,30,2004,0,14346025,1.042947,6.289872,0,0.530864,126162692,33.333332
...,...,...,...,...,...,...,...,...,...,...
Barry Zito,31,2009,0,22045868,1.026738,2.444635,0,0.543210,98451427,42.857143
Barry Zito,32,2010,0,21690091,1.065954,1.060084,0,0.567901,114698178,57.142857
Barry Zito,33,2011,0,21026387,0.286988,-2.264907,2,0.530864,134339671,71.428570


In [56]:
import numpy as np
from sklearn import linear_model
import statsmodels.api as sm

    In RStudio we ran regression anlayses on different combinations and interactions of variables with the target war variable. The lowest AIC value is represented by the total combination regressionn anlaysis. Thus in our Linear Regression Analysis we are going to include all the variables in our dataset. 

In [66]:
# Create the X and y variables
X = MLB_df.drop(columns='war_psp', axis=1)
y = MLB_df[['war_psp']]

In [67]:
X

,age,year,league,player_salary,percent_season_played,num_dl_movements,team_win_percentage,team_payroll,percent_contract_complete
name,,,,,,,,,
Bobby Abreu,26,2000,0,4354977,0.994676,0,0.401235,69700927,33.333332
Bobby Abreu,27,2001,0,7193330,1.029782,0,0.530864,60144837,66.666664
Bobby Abreu,28,2002,0,9000338,1.001989,0,0.493827,82364470,100.000000
Bobby Abreu,29,2003,0,12643905,1.016617,0,0.530864,98344579,16.666668
Bobby Abreu,30,2004,0,14346025,1.042947,0,0.530864,126162692,33.333332
...,...,...,...,...,...,...,...,...,...
Barry Zito,31,2009,0,22045868,1.026738,0,0.543210,98451427,42.857143
Barry Zito,32,2010,0,21690091,1.065954,0,0.567901,114698178,57.142857
Barry Zito,33,2011,0,21026387,0.286988,2,0.530864,134339671,71.428570


In [68]:
y

,war_psp
name,
Bobby Abreu,6.213081
Bobby Abreu,5.039903
Bobby Abreu,5.818425
Bobby Abreu,5.282226
Bobby Abreu,6.289872
...,...
Barry Zito,2.444635
Barry Zito,1.060084
Barry Zito,-2.264907


In [69]:
LR = linear_model.LinearRegression()
LR.fit(X, y)

print('Intercept: \n', LR.intercept_)
print('Coefficients: \n', LR.coef_)

Intercept: 
 [117.47242132]
Coefficients: 
 [[-1.19874977e-01 -5.75613409e-02 -8.94802570e-02  4.83269410e-08
   1.54960467e+00 -2.39776410e-01  7.29583295e+00 -1.41903211e-09
  -4.65463888e-03]]


In [71]:
X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)

print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:                war_psp   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     49.68
Date:                Sat, 06 Mar 2021   Prob (F-statistic):           8.91e-83
Time:                        20:44:33   Log-Likelihood:                -5506.0
No. Observations:                2385   AIC:                         1.103e+04
Df Residuals:                    2375   BIC:                         1.109e+04
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 